# Foundations of AI & ML
## Session 05
### HomeWork
### Lab

**Objectives:** Using the training data estimate the regression coefficients using maximum likelihood.
Refer http://www.machinegurning.com/rstats/logreg_math/ for detailed Problem Description.

**DataSet Description** This data approach student achievement in secondary education of two Portuguese schools. The data attributes include student grades, demographic, social and school related features) and it was collected by using school reports and questionnaires. We are going to work on the Maths DataSet.

**Attribute Information**

1 school - student's school (binary: 'GP' - Gabriel Pereira or 'MS' - Mousinho da Silveira) 

2 sex - student's sex (binary: 'F' - female or 'M' - male) 

3 age - student's age (numeric: from 15 to 22) 

4 address - student's home address type (binary: 'U' - urban or 'R' - rural) 

5 famsize - family size (binary: 'LE3' - less or equal to 3 or 'GT3' - greater than 3) 

6 Pstatus - parent's cohabitation status (binary: 'T' - living together or 'A' - apart) 

7 Medu - mother's education (numeric: 0 - none, 1 - primary education (4th grade), 2 â€“ 5th to 9th grade, 3 â€“ secondary education or 4 â€“ higher education) 

8 Fedu - father's education (numeric: 0 - none, 1 - primary education (4th grade), 2 â€“ 5th to 9th grade, 3 â€“ secondary education or 4 â€“ higher education) 

9 Mjob - mother's job (nominal: 'teacher', 'health' care related, civil 'services' (e.g. administrative or police), 'at_home' or 'other') 

10 Fjob - father's job (nominal: 'teacher', 'health' care related, civil 'services' (e.g. administrative or police), 'at_home' or 'other') 

11 reason - reason to choose this school (nominal: close to 'home', school 'reputation', 'course' preference or 'other') 

12 guardian - student's guardian (nominal: 'mother', 'father' or 'other') 

13 traveltime - home to school travel time (numeric: 1 - <15 min., 2 - 15 to 30 min., 3 - 30 min. to 1 hour, or 4 - >1 hour) 

14 studytime - weekly study time (numeric: 1 - <2 hours, 2 - 2 to 5 hours, 3 - 5 to 10 hours, or 4 - >10 hours)

15 failures - number of past class failures (numeric: n if 1<=n<3, else 4) 

16 schoolsup - extra educational support (binary: yes or no) 

17 famsup - family educational support (binary: yes or no) 

18 paid - extra paid classes within the course subject (Math) (binary: yes or no) 

19 activities - extra-curricular activities (binary: yes or no) 

20 nursery - attended nursery school (binary: yes or no) 

21 higher - wants to take higher education (binary: yes or no) 

22 internet - Internet access at home (binary: yes or no) 

23 romantic - with a romantic relationship (binary: yes or no) 

24 famrel - quality of family relationships (numeric: from 1 - very bad to 5 - excellent) 

25 freetime - free time after school (numeric: from 1 - very low to 5 - very high) 

26 goout - going out with friends (numeric: from 1 - very low to 5 - very high) 

27 Dalc - workday alcohol consumption (numeric: from 1 - very low to 5 - very high) 

28 Walc - weekend alcohol consumption (numeric: from 1 - very low to 5 - very high) 

29 health - current health status (numeric: from 1 - very bad to 5 - very good) 

30 absences - number of school absences (numeric: from 0 to 93) 

31 G1 - first period grade (numeric: from 0 to 20) 

32 G2 - second period grade (numeric: from 0 to 20) 

32 G3 - final grade (numeric: from 0 to 20, output target)

In [ ]:
import pandas as pd

In [ ]:
def convert(s):
    value = s.strip().lower()
    if value in ["gp", "f", "u", "gt3","a","mother","yes","at_home","pass"]:
        return 1
    if value in ["ms", "m", "r", "le3","t","father","no","services","fail"]:
        return 0
    if value in ["health","course"]:
        return 2
    if value in ["teacher","reputation"]:
        return 3
    return -1

data = pd.read_csv("../Datasets/newstudentdata.csv",converters={'school':convert, 'sex':convert, 'address':convert, 'famsize':convert, 'Pstatus':convert,
       'Mjob':convert, 'Fjob':convert, 'reason':convert, 'guardian':convert,
       'schoolsup':convert, 'famsup':convert, 'paid':convert, 'activities':convert,
       'nursery':convert, 'higher':convert, 'internet':convert, 'romantic':convert,'grade':convert})

In [ ]:
data.head()

split into training and test

In [ ]:
import numpy as np
import random
TRAIN_TEST_RATIO = 0.85
picker = list(range(data.shape[0]))
# random.shuffle(picker)       ### randomly shuffle the data
trainMax = int(len(picker) * TRAIN_TEST_RATIO)
train_data = []
test_data = []
train_labels = []
test_labels = []
for pick in picker[:trainMax]:
    train_data.append(list(data.values[pick][1:-1]))         ### select 80% of data to be used as training set
    train_labels.append((data.values[pick][-1]))
for pick in picker[trainMax:]:
    test_data.append(list(data.values[pick][1:-1]))       ### select 20% of data to be used as test set
    test_labels.append((data.values[pick][-1]))
num_feat = len(train_data[0])    ## number of features to train

train_data=np.array(train_data)
train_labels=np.array(train_labels)
train_labels = np.reshape(train_labels,(train_data.shape[0],1))
test_data=np.array(test_data)
test_labels=np.array(test_labels)
test_labels=np.reshape(test_labels,(test_data.shape[0],1))
print(num_feat)
print(train_data.shape, train_labels.shape)
print(train_data[0])
print(train_labels[0])


define logistic function and sgd

In [ ]:
import numpy as np
def logf(a, x):
    return 1.0 / (1.0 + np.exp(-((x @ a))))

def dlogf(a, x):
    return logf(a, x) * (1 - logf(a, x))
##
## The derivative of the logistic function is f * (1 - f)
##
def one_step(x, y, a, eta):
    x=np.hstack( (x,np.ones((x.shape[0],1))) )
    ypred = logf(a, x)
    error = np.asscalar(sum((y - ypred)**2))
    delta_a = sum((y - ypred) * ypred * (1 - ypred) * x)
    delta_a = np.reshape(delta_a, (delta_a.shape[0],1))
    a = a + (delta_a * eta)
    return a, error/y.shape[0]

do batch gradient descent to estimate coefficients

In [ ]:
a = np.zeros((num_feat+1,1))
eta = 0.002
for times in range(50000):
    a, error = one_step(train_data, train_labels, a, eta)
    if times%1000==0:
        print(error)

0.24793505019958695
0.1957189879307379
0.18577593186149366
0.20820371098349566
0.24973393832204593
0.18283058516374864
0.2013077001304799
0.1851025423558289
0.18697769560917862
0.18760383826033816
0.18187670799608507
0.1864335801557109
0.1841794382219442
0.18189380362710844
0.19104939443238822
0.18815784888864343
0.19485046776678827


get accuracy 

In [ ]:
test_data_append=np.hstack( (test_data,np.ones((test_data.shape[0],1))) )
pred_labels = np.round(logf(a,test_data_append))
print(pred_labels.shape, test_labels.shape)
#print((pred_labels[:,0]==test_labels[:,0]))

In [ ]:
results = []
for i in range(len(pred_labels)):
    results.append(pred_labels[i]==test_labels[i])

In [ ]:
results.count(True)/len(pred_labels)

**Excerise 1 :**  Plot the Precision-Recall Curve.

In [ ]:
from sklearn.metrics import precision_recall_curve
precision, recall, thresholds = precision_recall_curve(test_labels, pred_labels)
print(precision,recall,thresholds)

import matplotlib.pyplot as plt
plt.plot(precision,recall)

**Excerise 2:** Vary the above ratio between Training and Testing.Verify the effect if any on the prediction accuracy.

In [ ]:
def excerise2(TRAIN_TEST_RATIO):
    picker = list(range(data.shape[0]))
    # random.shuffle(picker)       ### randomly shuffle the data
    trainMax = int(len(picker) * TRAIN_TEST_RATIO)
    train_data = []
    test_data = []
    train_labels = []
    test_labels = []
    for pick in picker[:trainMax]:
        train_data.append(list(data.values[pick][1:-1]))         ### select 80% of data to be used as training set
        train_labels.append((data.values[pick][-1]))
    for pick in picker[trainMax:]:
        test_data.append(list(data.values[pick][1:-1]))       ### select 20% of data to be used as test set
        test_labels.append((data.values[pick][-1]))
    num_feat = len(train_data[0])    ## number of features to train

    train_data=np.array(train_data)
    train_labels=np.array(train_labels)
    train_labels = np.reshape(train_labels,(train_data.shape[0],1))
    test_data=np.array(test_data)
    test_labels=np.array(test_labels)
    test_labels=np.reshape(test_labels,(test_data.shape[0],1))

    a = np.zeros((num_feat+1,1))
    eta = 0.002
    for times in range(50000):
        a, error = one_step(train_data, train_labels, a, eta)
        #if times%1000==0:
            #print(error)
    
    test_data_append=np.hstack( (test_data,np.ones((test_data.shape[0],1))) )
    pred_labels = np.round(logf(a,test_data_append))
    print(pred_labels.shape, test_labels.shape)
    
    results = []
    for i in range(len(pred_labels)):
        results.append(pred_labels[i]==test_labels[i])
    
    return results.count(True)/len(pred_labels)
    

In [ ]:
ratio = []
acc = []
for i in range(1,10):
    print(i)
    r = 0.1*i
    ratio.append(r)
    acc.append(excerise2(r))

In [ ]:
plt.xlabel("Train-Test Ratio")
plt.ylabel("Accuracy")
plt.plot(ratio, acc)